In [ ]:
!pip install openai==0.28

In [ ]:
# prompt: load df using pandas

import pandas as pd
data = pd.read_csv('/content/bipole_train_1000.csv')
data.head()



,comment_text,label,old_id,id
0,"PERSON apparently thinks it is ""dangerous"" to ...",biased,5681508,1
1,I guess the Orange One didn't feel as though h...,biased,5875881,14
2,I think the consensus is that The Cherokee peo...,biased,904314,16
3,You mad bro'...??\nWell you're not alone...don...,biased,5954550,17
4,What is to stop the chief from selling native ...,biased,5115499,25


In [ ]:
data.drop(['old_id', 'id'], axis=1, inplace=True)
data

,comment_text,label
0,"PERSON apparently thinks it is ""dangerous"" to ...",biased
1,I guess the Orange One didn't feel as though h...,biased
2,I think the consensus is that The Cherokee peo...,biased
3,You mad bro'...??\nWell you're not alone...don...,biased
4,What is to stop the chief from selling native ...,biased
...,...,...
995,"For example PERSON, the Conservative governmen...",unbiased
996,"7,700 premature deaths per year. This does not...",unbiased
997,Are you calling North Korea a great nation?,unbiased
998,You should also carry private liability insura...,unbiased


In [ ]:
print('Total class''-->',data['label'].nunique(), "\n","\n", data['label'].value_counts())


Total class--> 2 
 
 biased      500
unbiased    500
Name: label, dtype: int64


In [ ]:
import pandas as pd

labels = data['label']
texts = data['comment_text']
df = pd.DataFrame(zip(texts, labels), columns = ['prompt','completion'])
df.head()

,prompt,completion
0,"PERSON apparently thinks it is ""dangerous"" to ...",biased
1,I guess the Orange One didn't feel as though h...,biased
2,I think the consensus is that The Cherokee peo...,biased
3,You mad bro'...??\nWell you're not alone...don...,biased
4,What is to stop the chief from selling native ...,biased


In [ ]:
df.to_json("bipole.jsonl", orient='records', lines=True)

In [ ]:
import openai


In [ ]:
!openai tools fine_tunes.prepare_data -f bipole.jsonl -q

Analyzing...

- Your file contains 1000 prompt-completion pairs
- Based on your data it seems like you're trying to fine-tune a model for classification
- For classification, we recommend you try one of the faster and cheaper models, such as `ada`
- For classification, you can estimate the expected model performance by keeping a held out dataset, which is not used for training
- There are 1 duplicated prompt-completion sets. These are rows: [251]
- Your data does not contain a common separator at the end of your prompts. Having a separator string appended to the end of the prompt makes it clearer to the fine-tuned model where the completion should begin. See https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset for more detail and examples. If you intend to do open-ended generation, then you should leave the prompts empty
- The completion should start with a whitespace character (` `). This tends to produce better results due to the tokenization we use. See https:/

In [ ]:
import openai



response = openai.File.create(
  file=open('/content/bipole_prepared_train.jsonl'),
  purpose='fine-tune'
)
file_id = response['id']


In [ ]:
fine_tune_response = openai.FineTune.create(
  training_file=file_id,
  model="davinci",
  n_epochs=10,
  batch_size=4
)
fine_tune_id = fine_tune_response['id']


In [ ]:
status = openai.FineTune.retrieve(id=fine_tune_id)
print(status)

{
  "object": "fine-tune",
  "id": "ft-7CYH7zJ0AAxrvoPvIPqRInm4",
  "hyperparams": {
    "n_epochs": 10,
    "batch_size": 4,
    "prompt_loss_weight": 0.01,
    "learning_rate_multiplier": 0.1
  },
  "organization_id": "org-4yQHWW6ba2Ck0rHchmD9THQl",
  "model": "davinci",
  "training_files": [
    {
      "object": "file",
      "id": "file-mn7I28lygETCSqHig0pk20qi",
      "purpose": "fine-tune",
      "filename": "file",
      "bytes": 269321,
      "created_at": 1701708978,
      "status": "processed",
      "status_details": null
    }
  ],
  "validation_files": [],
  "result_files": [
    {
      "object": "file",
      "id": "file-qsujVkH69iBUdvNKf9qvSNdB",
      "purpose": "fine-tune-results",
      "filename": "compiled_results.csv",
      "bytes": 91019,
      "created_at": 1701711172,
      "status": "processed",
      "status_details": null
    }
  ],
  "created_at": 1701708986,
  "updated_at": 1701711172,
  "status": "succeeded",
  "fine_tuned_model": "davinci:ft-personal-2

In [ ]:
import openai

# openai.api_key = 'sk-v6XTak5yuY1D9mOqRCLQT3BlbkFJXXs1dgmuQVoK'  # Replace with your actual API key

response = openai.Completion.create(
    model='davinci:ft-personal-2023-12-04-17-32-50',  #davinci:ft-personal-2023-12-04-14-26-46
    prompt='''\nYou will be given a sentence and you have to classify its label. The labels are
    'biased',
    'unbiased',
  Please try to classify the sentence as accurately as possible.
    The sentence is:
    Sentence = That's the ticket. Take this leaky ship that Doltan and PERSON built and steered toward the rocks and blow a great big NDP hole in the hull. Ontario will sink like a stone, but at least we'll be saved from the crash.\nLabel: ''',  #demo1 gave wromg
    # Sentence = Organised and paid protestors sway me not one bit. Well they might make me think on moving off in the other direction. ''', #demo2 unbiased gave correct
    # Sentence = You've never read Dr. PERSON's paper, and I don't believe you've honestly watched this film either. You're a yellow journalist. Take the piss somewhere else.	.	\nLabel: ''', #demo3 biased gave correct
    # Sentence = Unfortunately, mark my words, but your incredible political delusions will come back to roost when you can't get water and food in said disaster and then you will be faced with the reality that PERSON doesn't give a rip who you are too.\nLabel: ''', #demo4 unbiased gave wrong
    # Sentence = None of what you posted is what I said. Here is what I said: "Muslims aren't trying to turn America into a theocracy, let alone a Muslim theocracy. PERSON 'Christians' want a theocracy so bad, they wear it on their sleeve. (N.B. the Pence quote in the article. .\nLabel: ''', #demo5 biased gave correct
    max_tokens= 5
)

# Define the valid labels
valid_labels = [
    'biased', 'unbiased'
]

# Extract the response text
response_text = response.choices[0].text.strip()



In [ ]:
response_text

'biased . . . .'

In [ ]:
# curie:ft-personal-2023-12-01-21-23-26



---


#TESTING FOR COMPLETE DATASET



---



In [ ]:
import pandas as pd

test = pd.read_csv('bipole_test_set.csv')

test

,comment_text,label,old_id,id
0,That's the ticket. Take this leaky ship that ...,unbiased,5952388,83037
1,"When I read ""curriculum emphasizes group probl...",unbiased,5872499,13709
2,Organised and paid protestors sway me not one ...,unbiased,5577925,43383
3,Houston's response was far better than that of...,biased,5907356,82926
4,It says a lot about PERSON and PERSON when the...,unbiased,6013182,49478
...,...,...,...,...
295,"""President PERSON freezes Federal Government h...",unbiased,887248,68741
296,Huh? PERSON? Your comment seems really forced....,biased,5368813,15820
297,(Aight...aight. What about them bald headed bi...,biased,none,25296
298,"In general, the buying public far too often lo...",unbiased,5567031,82116


In [ ]:
test.drop(['old_id', 'id'], axis=1, inplace=True)
test

,comment_text,label
0,That's the ticket. Take this leaky ship that ...,unbiased
1,"When I read ""curriculum emphasizes group probl...",unbiased
2,Organised and paid protestors sway me not one ...,unbiased
3,Houston's response was far better than that of...,biased
4,It says a lot about PERSON and PERSON when the...,unbiased
...,...,...
295,"""President PERSON freezes Federal Government h...",unbiased
296,Huh? PERSON? Your comment seems really forced....,biased
297,(Aight...aight. What about them bald headed bi...,biased
298,"In general, the buying public far too often lo...",unbiased


In [ ]:
# import pandas as pd
# import openai
# from sklearn.metrics import classification_report

# # Extract necessary columns (assuming 'comment_text' contains the text data)
# test_texts = test['comment_text']

# # Prepare data for inference (creating prompts)
# prompts = [f"You will be given a sentence and you have to classify its label. The labels are 'biased' and 'unbiased'. Please try to classify the sentence as accurately as possible. The sentence is: Sentence = {text} Label: " for text in test_texts]




# # Make predictions on the test data
# predicted_labels = []
# model = 'davinci:ft-personal-2023-12-04-17-32-50'
# for prompt in prompts:
#     response = openai.Completion.create(
#         model=model,
#         prompt=prompt,
#         max_tokens=5
#     )
#     response_text = response.choices[0].text.strip()
#     predicted_labels.append(response_text)

# # Add predicted labels to test data
# test['predicted_label'] = predicted_labels



# # # Create a classification report
# # print(classification_report(test['label'], test['predicted_label']))


# # import re

# # # Regular expressions for extracting "biased" or "unbiased" from the predicted labels
# # biased_pattern = re.compile(r'biased', re.IGNORECASE)
# # unbiased_pattern = re.compile(r'unbiased', re.IGNORECASE)

# # # Function to categorize labels using regex
# # def categorize_label(label):
# #     if re.search(biased_pattern, label):
# #         return 'biased'
# #     elif re.search(unbiased_pattern, label):
# #         return 'unbiased'
# #     else:
# #         return 'unknown'  # Change or handle other cases if necessary

# # # Filter and categorize predicted labels
# # predicted_labels_filtered = [categorize_label(label) for label in predicted_labels]

# # # Add filtered labels to test data
# # test['predicted_label_filtered'] = predicted_labels_filtered

# # # Create a classification report
# # print(classification_report(test['label'], test['predicted_label_filtered']))


In [ ]:
unknown_samples = test['comment_text']

# Simplified prompt
prompt_template = """
Look at the text below and decide if it is 'Biased' or 'Unbiased'. Use these simple definitions to help you:

- 'Biased': Information that seems to favor one side or view, not balanced.
- 'Unbiased': Information that is fair to all sides, balanced and neutral.

Kindly rethink before giving your prediction and classify with utmost logic and careful thinking. A 'biased' sentence generally indicates a sentence which may contain race ethinicity, ageism, gender, occupation, religion, LGBTQ bias. Whereas an 'unbiased' sentence are plain and simple sentences. Here no such biases are reflected.

Text to Classify:
{}

Classification (Choose 'biased' or 'unbiased' only):
"""

result = []

for sample in unknown_samples:
    prompt = prompt_template.format(sample)

    # Adjusted hyperparameters for improved responses
    response = openai.Completion.create(
        model="davinci:ft-personal-2023-12-04-17-32-50",
        prompt=prompt,
        temperature=0.6,  # More deterministic responses = 0.6
        max_tokens=5,      # Restricting to only necessary tokens
        top_p=0.8,           # Considering all possible completions
        frequency_penalty=0,  # Avoiding penalization for frequent words
        presence_penalty=0    # Not focusing on new concepts
    )

    label = response['choices'][0]['text'].strip()
    result.append(label)

print(result)


['###\n\n biased', '###\n\n biased', '###\n\n biased', '###\n\n biased', '###\n\n biased', '###\n\n biased', '###\n\n unbiased biased', '###\n\n biased', '###\n\n biased', '###\n\n unbiased biased', '###\n\n biased', '###\n\n biased biased', '###\n\n biased', '###\n\n biased biased', '###\n\n biased', '###\n\n biased', '###\n\n biased', '###\n\n biased biased', '###\n\n biased', '###\n\n biased', '###\n\n unbiased biased', '###\n\n biased', '###\n\n unbiased biased', '###\n\n biased', '###\n\n biased', 'Reasoning:', '###\n\n biased', '###\n\n biased', '###\n\n biased', '###\n\n biased unbiased', '###\n\n biased', '###\n\n biased', '###\n\n biased', '###\n\n unbiased biased', '###\n\n biased', '###\n\n biased', '###\n\n biased', '###\n\n unbiased biased', '###\n\n biased', '###\n\n biased', '###\n\n biased', '###\n\n biased biased', '###\n\n biased', '###\n\n biased', '###\n\n biased', '###\n\n unbiased biased', '###\n\n biased', '###\n\n biased', '###\n\n biased biased', '###\n\n biase

In [ ]:
results

In [ ]:
# List of input strings
input_strings = ['###\n\n biased\n\n biased',
 '###\n\n unbiased biased\n\n biased',
 '###\n\n unbiased biased\n\n biased',
 '###\n\n biased\n\n biased',
 '###\n\n biased\n\n biased',
 '###\n\n unbiased biased\n\n biased',
 '###\n\n biased\n\n unbiased',
 '###\n\n biased\n\n\n biased',
 '###\n\n biased\n\n biased',
 '###\n\n unbiased biased',
 '###\n\n biased\n\n biased',
 'PERSON\n\n###\n\n biased',
 'My Prediction:',
 '###\n\n biased\n\n biased',
 '###\n\n biased\n\n unbiased',
 '###\n\n unbiased biased\n\n biased',
 '"Unbiased"',
 '###\n\n biased\n\n unbiased',
 'A. Biased',
 '###\n\n biased\n\n biased',
 '###\n\n biased\n\n###',
 '###\n\n biased\n\n biased',
 '###\n\n biased biased biased biased',
 '###\n\n biased biased biased biased',
 '###\n\n biased\n\n biased',
 '###\n\n biased\n\n unbiased',
 '###\n\n biased\n\n biased',
 '###\n\n unbiased biased\n\n biased',
 '###\n\n unbiased biased\n\n biased',
 '###\n\n biased\n\n unbiased',
 'biased\n\nunbiased',
 '###\n\n biased\n\n biased',
 '###\n\n biased\n\n\n biased',
 '###\n\n unbiased biased\n\n biased',
 '###\n\n biased\n\n unbiased',
 '###\n\n biased\n\n biased',
 '###\n\n biased\n\n biased',
 '###\n\n biased\n\n biased',
 '###\n\n biased\n\n biased',
 '###\n\n biased\n\n biased',
 '###\n\n unbiased biased\n\n biased',
 '###\n\n biased\n\n unbiased',
 '###\n\n unbiased biased\n\n biased',
 '###\n\n biased\n\n biased',
 '###\n\n biased\n\n unbiased',
 '###\n\n unbiased biased biased biased',
 '###\n\n unbiased biased',
 '###\n\n unbiased biased\n\n biased',
 '###\n\n biased biased biased biased',
 '###\n\n unbiased biased\n\n biased',
 '###\n\n biased\n\n###',
 '###\n\n biased\n\n###',
 '###\n\n unbiased biased\n\n biased',
 '###\n\n biased\n\n biased',
 '###\n\n unbiased unbiased\n\n biased',
 '###\n\n unbiased biased',
 '###\n\n unbiased biased\n\n biased',
 '###\n\n unbiased biased biased biased',
 '###\n\n biased\n\n###',
 '###\n\n biased\n\n unbiased',
 '###\n\n biased\n\n biased',
 '###\n\n biased\n\n biased',
 '###\n\n biased\n\n biased',
 '###\n\n unbiased biased\n\n biased',
 '###\n\n unbiased biased\n\n biased',
 '###\n\n biased\n\n###',
 '###\n\n biased\n\n biased',
 '###\n\n biased\n\n biased',
 '###\n\n biased\n\n biased',
 '###\n\n unbiased biased\n\n biased',
 '###\n\n unbiased biased\n\n biased',
 '###\n\n biased\n\n biased',
 '###\n\n biased\n\n\n5',
 '###\n\n biased\n\n biased',
 '###\n\n biased\n\n###',
 '###\n\n biased\n\n biased',
 '###\n\n biased\n\n biased',
 '###\n\n biased\n\n biased',
 '###\n\n biased\n\n\n biased',
 '###\n\n biased\n\n biased',
 '###\n\n unbiased biased\n\n biased',
 '###\n\n biased\n\n biased',
 '###\n\n unbiased biased biased biased',
 '###\n\n biased\n\n unbiased',
 'a) Biased',
 '###\n\n biased\n\n unbiased',
 '###\n\n biased\n\n unbiased',
 '###\n\n biased\n\n biased',
 '###\n\n biased\n\n biased',
 '###\n\n biased\n\n biased',
 '###\n\n unbiased unbiased\n\n biased',
 '###\n\n biased\n\n###',
 'A) Unbiased',
 '###\n\n biased\n\n biased',
 '###\n\n biased\n\n biased',
 '###\n\n biased\n\n biased',
 '###\n\n biased\n\n\n biased',
 '###\n\n biased\n\n biased',
 '###\n\n biased\n\n###',
 '###\n\n biased\n\n biased',
 '###\n\n biased\n\n biased',
 '###\n\n biased\n\n unbiased',
 '###\n\n biased\n\n biased',
 '###\n\n biased\n\n biased',
 '###\n\n biased\n\n biased',
 '###\n\n biased\n\n biased',
 '###\n\n biased\n\n unbiased',
 '###\n\n biased biased biased biased',
 '###\n\n unbiased unbiased unbiased unbiased',
 '###\n\n biased\n\n biased',
 '###\n\n biased\n\n biased',
 '###\n\n biased\n\n biased',
 '###\n\n biased\n\n biased',
 '###\n\n unbiased biased\n\n biased',
 '###\n\n biased\n\n###',
 '###\n\n biased\n\n unbiased',
 '###\n\n biased\n\n unbiased',
 '###\n\n unbiased biased\n\n biased',
 '###\n\n unbiased\n\n biased',
 '###\n\n biased\n\n biased',
 '###\n\n biased\n\n biased',
 '###\n\n unbiased biased\n\n biased',
 '###\n\n biased\n\n unbiased',
 '###\n\n biased\n\n biased',
 '###\n\n unbiased biased\n\n biased',
 '###\n\n unbiased biased biased biased',
 '###\n\n unbiased biased\n\n biased',
 '###\n\n unbiased biased\n\n biased',
 '###\n\n biased\n\n biased',
 '###\n\n biased biased biased biased',
 '###\n\n biased\n\n biased',
 '###\n\n biased\n\n unbiased',
 '###\n\n biased\n\n biased',
 '###\n\n biased\n\n unbiased',
 '###\n\n biased\n\n biased',
 '###\n\n unbiased biased biased biased',
 '###\n\n biased\n\n biased',
 '###\n\n unbiased biased',
 '###\n\n biased\n\n biased',
 '###\n\n biased\n\n unbiased',
 '###\n\n biased\n\n unbiased',
 '###\n\n unbiased biased\n\n biased',
 '###\n\n unbiased unbiased unbiased unbiased',
 '###\n\n biased\n\n biased',
 '###\n\n biased\n\n biased',
 '###\n\n biased\n\n biased',
 '###\n\n biased\n\n biased',
 '###\n\n biased\n\n biased',
 '###\n\n unbiased biased',
 '###\n\n biased\n\n biased',
 '###\n\n unbiased biased',
 '###\n\n biased\n\n biased',
 '###\n\n biased\n\n unbiased',
 '###\n\n unbiased biased\n\n biased',
 '###\n\n biased\n\n biased',
 '###\n\n biased\n\n biased',
 '###\n\n unbiased biased\n\n biased',
 '###\n\n biased\n\n biased',
 '###\n\n unbiased biased\n\n biased',
 '###\n\n biased\n\n biased',
 '###\n\n biased\n\n biased',
 '###\n\n unbiased biased biased biased',
 '###\n\n biased\n\n\n###',
 '###\n\n biased\n\n biased',
 'PERSON is a stupid,',
 '###\n\n biased\n\n unbiased',
 '###\n\n unbiased\n\n biased',
 'Unbiased.\n\n###',
 '###\n\n unbiased unbiased unbiased unbiased',
 '###\n\n biased\n\n biased',
 '###\n\n biased\n\n###',
 '###\n\n unbiased\n\n biased',
 '###\n\n unbiased biased',
 '###\n\n biased\n\n biased',
 '###\n\n biased biased biased biased',
 '###\n\n unbiased\n\n biased',
 '###\n\n unbiased biased\n\n biased',
 'A. B.',
 '###\n\n biased\n\n biased',
 '###\n\n unbiased biased\n\n biased',
 '###\n\n unbiased biased biased',
 '###\n\n unbiased biased\n\n biased',
 '###\n\n biased\n\n biased',
 '###\n\n biased\n\n###',
 '###\n\n biased###\n\n biased',
 '###\n\n biased\n\n biased',
 '###\n\n unbiased biased',
 '###\n\n biased\n\n biased',
 '###\n\n biased\n\n biased',
 '###\n\n unbiased biased',
 '###\n\n biased\n\n biased',
 'Biased.\n\nMy comments',
 '###\n\n unbiased unbiased\n\n biased',
 '###\n\n unbiased biased\n\n biased',
 '###\n\n biased\n\n biased',
 '###\n\n biased\n\n unbiased',
 'What is your prediction?',
 '###\n\n unbiased biased\n\n biased',
 '###\n\n biased\n\n biased',
 '###\n\n biased\n\n unbiased',
 '###\n\n unbiased biased\n\n biased',
 '###\n\n biased\n\n biased',
 '###\n\n biased\n\n biased',
 '###\n\n biased biased biased biased',
 '###\n\n unbiased biased\n\n biased',
 '###\n\n biased\n\n biased',
 '###\n\n biased\n\n biased',
 '###\n\n biased\n\n unbiased',
 '###\n\n biased\n\n biased',
 'Your Name:\n\nYour',
 'A. Biased',
 '###\n\n biased\n\n###',
 '###\n\n biased\n\n biased',
 '###\n\n biased\n\n biased',
 '###\n\n biased\n\n unbiased',
 '###\n\n unbiased biased\n\n biased',
 '###\n\n biased\n\n###',
 '###\n\n biased\n\n unbiased',
 '###\n\n biased\n\n unbiased',
 '###\n\n biased\n\n biased',
 '###\n\n biased\n\n unbiased',
 'Biased -',
 '###\n\n unbiased biased\n\n biased',
 '###\n\n unbiased\n\n biased',
 '###\n\n biased\n\n biased',
 '###\n\n unbiased biased\n\n biased',
 '###\n\n biased\n\n\n biased',
 '###\n\n biased\n\n biased',
 '###\n\n biased\n\n unbiased',
 '###\n\n biased\n\n biased',
 '###\n\n biased\n\n biased',
 '###\n\n unbiased biased\n\n biased',
 '###\n\n biased\n\n biased',
 '###\n\n biased\n\n unbiased',
 '###\n\n biased\n\n unbiased',
 '###\n\n biased\n\n biased',
 'Biased\n\nUnbiased',
 '###\n\n biased\n\n###',
 '###\n\n unbiased biased\n\n biased',
 '###\n\n biased\n\n unbiased',
 '###\n\n unbiased biased\n\n biased',
 '###\n\n biased\n\n biased',
 '###\n\n unbiased biased\n\n biased',
 '###\n\n biased\n\n biased',
 'a) Unbiased',
 '###\n\n biased\n\n unbiased',
 '###\n\n unbiased biased\n\n biased',
 'A. Bias',
 '###\n\n unbiased unbiased\n\n biased',
 '###\n\n biased\n\n###',
 '###\n\n biased\n\n\n biased',
 'A) Biased',
 '###\n\n biased\n\n###',
 '###\n\n biased\n\n biased',
 '###\n\n biased\n\n biased',
 '###\n\n biased\n\n unbiased',
 '###\n\n biased\n\n unbiased',
 '###\n\n biased\n\n unbiased',
 '###\n\n biased\n\n###',
 '###\n\n biased\n\n biased',
 '###\n\n biased biased biased biased',
 '###\n\n unbiased biased\n\n biased',
 '###\n\n biased\n\n biased',
 '###\n\n biased\n\n###',
 '###\n\n biased\n\n biased',
 '###\n\n unbiased biased\n\n biased',
 'A) Unbiased',
 'Unbiased.\n\n###',
 '###\n\n unbiased biased\n\n biased',
 '###\n\n biased\n\n biased',
 '###\n\n biased\n\n biased',
 '###\n\n biased\n\n unbiased',
 '###\n\n biased\n\n biased',
 '###\n\n biased\n\n biased',
 '###\n\n biased\n\n biased',
 '###\n\n unbiased biased\n\n biased',
 '###\n\n biased\n\n biased',
 '###\n\n biased\n\n###',
 'Your Answer:\n\n biased',
 '###\n\n biased biased biased biased',
 '###\n\n biased\n\n###',
 '###\n\n biased\n\n\n biased',
 '###\n\n unbiased\n\n biased',
 '###\n\n unbiased biased\n\n biased',
 '###\n\n biased\n\n unbiased',
 '###\n\n biased\n\n###',
 '###\n\n biased\n\n\n###',
 '###\n\n unbiased biased\n\n biased',
 '###\n\n biased\n\n biased',
 '__ biased\n\n__ unbiased',
 '###\n\n unbiased biased\n\n biased',
 '###\n\n biased\n\n unbiased',
 '###\n\n biased\n\n biased',
 '###\n\n unbiased biased\n\n biased',
 '###\n\n biased\n\n biased',
 '###\n\n biased\n\n biased',
 '###\n\n biased, biased',
 '###\n\n biased\n\n biased',
 '###\n\n biased\n\n biased',
 '###\n\n biased\n\n biased']

# Function to extract the label
def extract_label(s):
    if 'unbiased' in s:
        return 'unbiased'
    elif 'biased' in s:
        return 'biased'
    else:
        return 'unbiased'  # Default label

# Extracting labels
extracted_labels = [extract_label(s) for s in input_strings]

# Printing the result
for label in extracted_labels:
    print(label)

In [ ]:
len(extracted_labels)

300

In [ ]:
print(classification_report(test['label'], extracted_labels))

              precision    recall  f1-score   support

      biased       0.58      0.64      0.61       150
    unbiased       0.60      0.54      0.57       150

    accuracy                           0.59       300
   macro avg       0.59      0.59      0.59       300
weighted avg       0.59      0.59      0.59       300



In [ ]:
type(results)

list

In [ ]:
predicted_labels

['Biased- Compelled',
 'Unbiased, Bi',
 '(bias negative or',
 '= biased; Unbiased',
 'Risk:',
 '= [unbiased',
 'How many people',
 'unBIASED Label:',
 'biased Label: unbiased',
 'Unbiased Label',
 '"This is not going',
 'Balanced  Biased',
 "Don't respond to",
 '"Bias"',
 'Observed behavior, you',
 'biased  unbiased',
 'biased\n\nIf you',
 'biased and unreasoned',
 'Biased  Unbiased',
 'Biased = An experienced',
 'Biased  Unbiased',
 '_______________\nDate:',
 'Biased, Unbiased',
 "Today's Bias Fact",
 '###\n\n biased',
 '_____ Perceived Label:',
 'Your Position:',
 'Label = Bias',
 'Unbiased = Class',
 'Unbiased:',
 'biased Label: Unbiased',
 'biased label:',
 'biased, or unbiased.',
 'Biased or Un',
 '###\n\n biased',
 'Unbiased\n\n###',
 'biased or unbiased.',
 'Bias\n\nNon',
 'biased LABEL =',
 'Unbiased  Biased',
 'Quinn asked',
 '_?',
 'Unbiased,',
 'biased or unbiased  Positive',
 'Remember:',
 'Biased Label: Un',
 '?',
 'NOTE: Please remember',
 'UNBIASED or BI',
 '',
 'Biased 